In [4]:
import pickle
import plotly.express as px
import plotly.io as pio
import matplotlib as plt
import os
from bertopic.representation import PartOfSpeech
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.backend import BaseEmbedder
from sentence_transformers import SentenceTransformer
import pandas as pd
import random 

In [5]:
wort = input("Enter the name of the variable: ")
# Name of the output folder and the file
output_folder_name = wort
# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_name):
    os.makedirs(output_folder_name)

csv_file_path = '/Users/davideventre/Desktop/telegram_daten/telegram_data.csv'

# Read the CSV file
# Get the number of rows in the DataFrame
if wort == 'me':
    df = pd.read_csv(csv_file_path, sep='\t', low_memory=False)
    text_content = df['text_content']
    text_date = df['text_date']
    text_source = df['text_source']
    text_id = df['text_id']
elif wort == 'te':
# Read the CSV file
    # Get the number of rows in the DataFrame
    df = pd.read_csv(csv_file_path, sep=',', low_memory=False)
    num_rows = len(df)
    random_indices = random.sample(range(num_rows), 1000)
    random_indices.sort()
    # Select specific columns and convert them to lists using the random indices
    text_content = df['message'].astype(str).tolist()
    text_date = df['date'].astype(str).tolist()
    text_source = df['channel_id'].astype(str).tolist()
    text_views= df['view_list'].astype(str).tolist()
    text_forwards = df['forwards'].astype(str).tolist()
    """
    text_content = df['message']
    text_id = df['date']
    text_source = df['channel_id']
    text_id = df['channel_id']
    """
    

In [6]:
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
representation_model = [KeyBERTInspired(top_n_words=10), MaximalMarginalRelevance(diversity=0.95)]
# umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', low_memory=False)
# nextneighbors: higher = more global, dimensionality of embedding (high or low no good cluster)
umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine')
# min cluster set higher than default for bigger clusters (def = 10)
hdbscan_model = HDBSCAN(min_cluster_size=100, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=3)# custom vectorizer, can later be changed to specific options. min_df = how many times a word must occur
vectorizer_model = CountVectorizer(min_df=8, ngram_range=(1, 3))
ctfidf_model = ClassTfidfTransformer()

topic_model = BERTopic(embedding_model=sentence_model, representation_model=representation_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, nr_topics='auto')

topics, probs = topic_model.fit_transform(text_content)
topic_model_distr = BERTopic().fit(text_content)
# Reduce outliers
new_topics = topic_model.reduce_outliers(text_content, topics)
# versions of dependencies and python used. loading and saving model => same dependencies and python
# saved in one version of Bertopic should not be loaded in others
#topic_model = BERTopic.load("telegram_model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

KeyboardInterrupt: 

In [ ]:

#visualization of the topics and their cluster
#Then, we can call .visualize_topics to create a 2D representation of your topics. The resulting graph is a plotly interactive graph which can be converted to HTML:
fig = topic_model.visualize_topics()
# Convert the figure to HTML
html = pio.to_html(fig)
html_file_path = os.path.join(output_folder_name, 'topic' + wort + '.html')
with open(html_file_path, 'w') as f:
    f.write(html)


fig0 = topic_model.visualize_barchart(top_n_topics=20, n_words=20)
# Convert the figure to HTML
html0 = pio.to_html(fig0)
html_file_path0 = os.path.join(output_folder_name, 'barchart_' + wort + '.html')
with open(html_file_path0, 'w') as f:
    f.write(html0)


In [ ]:
#hierarchical topic modeling
# hierarchical topic modeling
hierarchical_topics = topic_model.hierarchical_topics(text_content)
fig2 = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
html2 = pio.to_html(fig2)
html_file_path2 = os.path.join(output_folder_name, 'hierarchical_' + wort + '.html')
with open(html_file_path2, 'w') as f:
    f.write(html2)
#topics_to_merge = [[1, 2],[3, 4]]
#topic_model.merge_topics(docs, topics_to_merge)"""

In [ ]:

# Similarity of topics with each other
heatmap = topic_model.visualize_heatmap()
html4 = pio.to_html(heatmap)
html_file_path3 = os.path.join(output_folder_name, 'heatmap_' + wort + '.html')
with open(html_file_path3, 'w') as f:
    f.write(html4)


In [ ]:
# Topics ordered by a specific class 
topics_per_class = topic_model.topics_per_class(text_content, classes=text_source)
visualize_class = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
html5 = pio.to_html(visualize_class)
html_file_path5 = os.path.join(output_folder_name, 'class_' + wort + '.html')
with open(html_file_path5, 'w') as f:
    f.write(html5)


In [ ]:
try:
    topics_over_time = topic_model.topics_over_time(text_content, text_date, nr_bins=20, datetime_format='%Y-%m-%d')
    fig1 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=25)
    html1 = pio.to_html(fig1)
    html_file_path1 = os.path.join(output_folder_name + 'dynamic' + '.html')
    with open(html_file_path1, 'w') as f:
        f.write(html1)
except ValueError:
    print('Class ValueError')
    

In [ ]:
docs = text_content
# Run the visualization with the original embeddings
visualize_docs = topic_model.visualize_documents(docs, embeddings=embeddings, hide_annotations=True, hide_document_hover=False, width = 2400, height = 1400)
html_file_path7 = os.path.join(output_folder_name, 'documents_' + wort + '.html')
html3 = pio.to_html(visualize_docs)
# Write the HTML content to the file
with open(html_file_path7, 'w') as f:
    f.write(html3)

In [ ]:

# 3
topic_info = topic_model.get_document_info(text_content)
# Assuming topic_info is a list of dictionaries with keys like 'document_id', 'topic_distribution', etc.
# Saving the data to a .txt file
topic_data = pd.DataFrame(topic_info)
csv_file_path1 = os.path.join(output_folder_name, 'list_' + wort + '.csv')
topic_data.to_csv(csv_file_path1, index=False)  # Set index=False to exclude row indices in the output

In [ ]:
topic_distr, _ = topic_model_distr.approximate_distribution(text_content)
distribution_vis = topic_model.visualize_distribution(topic_distr[1])
html_file_path8 = os.path.join(output_folder_name, 'distribution' + wort + '.html')
html8 = pio.to_html(distribution_vis)
with open(html_file_path8, 'w') as f:
    f.write(html8)